# 使用最近邻方法更新聚类 ID

**1）更新原理**

给定邻域 `eps`，若新加入 embedding 在某个簇的聚类中心的 `eps` 邻域范围内，则归入该簇；否则该 embedding 单独建簇。

**2）更新步骤**

初始化：

1. 用 dbscan 计算每个 embedding 样本的簇 ID
2. 对于 dbscan，当样本的簇 ID 为 -1 时，说明该样本为离群点。这时赋给该样本一个与现
3. 当样本的簇 ID 为 -1 时，为该样本生成一个与现有簇 ID 不重复的数，作为该样本的簇 ID
4. 计算每个簇的聚类中心
5. 创建两个 DataFrame底表（回溯数据用，不参与后续更新）：列名为 embedding, dbscan_id, cluster_id, cluster_center
    - 聚类 ID 表：列名为 embedding, cluster_id
    - 聚类中心表：列名为 cluster_id, cluster_center

更新：

1. 当有新 embedding 进入时，计算该 embedding 是否在任一聚类中心的 eps 邻域内
    - 如果在，让 embedding 加入该簇，并更新该簇的聚类中心
    - 如果不在，创建一个新簇，该 embedding 自己作为聚类中心
2. 更新两个 DataFrame 表


In [1]:
import os
import collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import utils

from typing import Dict, Set, Tuple, List
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_distances

CSV_PATH = './data'
SAMPLE_NUM = 7000

os.environ["LOKY_MAX_CPU_COUNT"] = "4"

if utils.in_jupyter():
    # 在 Jupyter 时 tqdm 的导入方式
    from tqdm.notebook import tqdm
else:
    # 在终端时 tqdm 的导入方式
    from tqdm import tqdm

In [2]:
# 将 csv 读入 DataFrame
train_csv_path = os.path.join(CSV_PATH, 'embed_label.csv')
df = utils.read_embedding_csv(csv_path=train_csv_path,
                              ebd_cols=['embeddings'])
len(df), len(set(df['labels'].tolist()))

(10000, 100)

In [3]:
df = df.drop(columns=['labels'])

手动添加一列 `item_id` 作为每个 embedding 的唯一标识。

In [4]:
df['item_id'] = [i for i in range(len(df))]
df = df[['item_id', 'embeddings']]
df

,item_id,embeddings
0,0,"[0.024523582309484482, -0.03633105754852295, 0..."
1,1,"[-0.002521098591387272, 0.022899063304066658, ..."
2,2,"[0.008400454185903072, -0.012612388469278812, ..."
3,3,"[-0.004734962247312069, -0.0035224033053964376..."
4,4,"[-0.021240245550870895, -0.03918471559882164, ..."
...,...,...
9995,9995,"[0.02526906132698059, 0.006334671750664711, 0...."
9996,9996,"[-0.0032427890691906214, 0.0032633657101541758..."
9997,9997,"[0.001930834841914475, -0.025012478232383728, ..."
9998,9998,"[0.025050941854715347, -0.017404677346348763, ..."


## 一、初始化

In [5]:
df['embeddings'][0].shape

(1408,)

In [6]:
train_df = df.head(SAMPLE_NUM).copy()

eps=0.1
clustering = DBSCAN(eps=eps, min_samples=3, metric='cosine').fit(train_df['embeddings'].tolist())
labels = clustering.labels_

In [7]:
len(labels), max(labels), len([1 for e in labels if e != -1])

(7000, 147, 1993)

In [8]:
labels_counter = collections.Counter(labels)
sorted_labels = sorted(labels_counter.items(), key=lambda e: e[1], reverse=True)
sorted_labels[:5]

[(-1, 5007), (10, 117), (17, 72), (8, 67), (6, 64)]

In [9]:
# 把 labels 作为 dbscan_id 写入 DataFrame 中
train_df['dbscan_id'] = labels
train_df

,item_id,embeddings,dbscan_id
0,0,"[0.024523582309484482, -0.03633105754852295, 0...",-1
1,1,"[-0.002521098591387272, 0.022899063304066658, ...",0
2,2,"[0.008400454185903072, -0.012612388469278812, ...",-1
3,3,"[-0.004734962247312069, -0.0035224033053964376...",-1
4,4,"[-0.021240245550870895, -0.03918471559882164, ...",1
...,...,...,...
6995,6995,"[0.013036987744271755, 0.004907825030386448, 0...",5
6996,6996,"[0.006075031124055386, 0.06972860544919968, 0....",-1
6997,6997,"[-0.0033710638526827097, 0.03442999720573425, ...",-1
6998,6998,"[-0.02391742914915085, 0.04698537290096283, 0....",100


In [10]:
# 新增一个列 cluster_id，为值为 -1 的类赋予 label

def id_generator(used_id_set: Set[int]):
    """生成未被使用的最小ID"""
    i = 0
    while True:
        while i in used_id_set:
            i += 1
        yield i
        i += 1

dbscan_ids = train_df['dbscan_id'].tolist()
gen = id_generator(set(dbscan_ids))

cluster_id = list()
for e in train_df['dbscan_id'].tolist():
    if e == -1:
        cluster_id.append(next(gen))
    else:
        cluster_id.append(e)

train_df['cluster_id'] = cluster_id
train_df.head(5)

,item_id,embeddings,dbscan_id,cluster_id
0,0,"[0.024523582309484482, -0.03633105754852295, 0...",-1,148
1,1,"[-0.002521098591387272, 0.022899063304066658, ...",0,0
2,2,"[0.008400454185903072, -0.012612388469278812, ...",-1,149
3,3,"[-0.004734962247312069, -0.0035224033053964376...",-1,150
4,4,"[-0.021240245550870895, -0.03918471559882164, ...",1,1


In [11]:
# 为每个 cluster_id 计算聚类中心
cluster_centers = list()
clstrid2center = dict()  # 字典，用于缓存 聚类 id -> 聚类中心
for e in train_df['cluster_id'].tolist():
    if e in clstrid2center.keys():
        # 从字典中读取
        cluster_center = clstrid2center[e]
    else:
        embeds = train_df[train_df["cluster_id"] == e]["embeddings"]
        cluster_center = np.mean(embeds, axis=0).tolist()
        clstrid2center[e] = cluster_center  # 存入字典

    cluster_centers.append(cluster_center)

train_df['cluster_center'] = cluster_centers
train_df.head(5)

,item_id,embeddings,dbscan_id,cluster_id,cluster_center
0,0,"[0.024523582309484482, -0.03633105754852295, 0...",-1,148,"[0.024523582309484482, -0.03633105754852295, 0..."
1,1,"[-0.002521098591387272, 0.022899063304066658, ...",0,0,"[-0.00018301361706107855, 0.022485706851714186..."
2,2,"[0.008400454185903072, -0.012612388469278812, ...",-1,149,"[0.008400454185903072, -0.012612388469278812, ..."
3,3,"[-0.004734962247312069, -0.0035224033053964376...",-1,150,"[-0.004734962247312069, -0.0035224033053964376..."
4,4,"[-0.021240245550870895, -0.03918471559882164, ...",1,1,"[-0.006647970941932206, -0.02852569787230875, ..."


在进行更新前，我们把底表拆成两张表。

- **聚类 ID 表**：`item_id`, `embedding` -> `cluster_id`
- **聚类中心表**：`cluster_id`  -> `cluster_center`

In [12]:
# 创建聚类 ID 表
cluster_id_df = train_df[['item_id', 'embeddings', 'cluster_id']]

duplicate_mask = cluster_id_df.duplicated(subset=['item_id'], keep=False)
if duplicate_mask.any():
    duplicate_ids = df.loc[duplicate_mask, 'item_id'].unique().tolist()
    raise ValueError( f"重复行数: {duplicate_mask.sum()}")

len(cluster_id_df)

7000

In [13]:
cluster_id_df.head(5)

,item_id,embeddings,cluster_id
0,0,"[0.024523582309484482, -0.03633105754852295, 0...",148
1,1,"[-0.002521098591387272, 0.022899063304066658, ...",0
2,2,"[0.008400454185903072, -0.012612388469278812, ...",149
3,3,"[-0.004734962247312069, -0.0035224033053964376...",150
4,4,"[-0.021240245550870895, -0.03918471559882164, ...",1


In [14]:
# 创建聚类中心表
cluster_center_tmp_df = train_df[['cluster_id', 'cluster_center']]
cluster_center_df = cluster_center_tmp_df.groupby('cluster_id').first().reset_index()

len(cluster_center_df)

5155

In [15]:
cluster_center_df.head(5)

,cluster_id,cluster_center
0,0,"[-0.00018301361706107855, 0.022485706851714186..."
1,1,"[-0.006647970941932206, -0.02852569787230875, ..."
2,2,"[0.003304169833427295, -0.008476288610836491, ..."
3,3,"[-0.0052974170277593656, 0.035157745331525804,..."
4,4,"[0.0017625399269774313, -0.019622230612230104,..."


## 二、更新

> **WARNING:** 由于 DBSCAN 是一种基于密度成簇的方法，其边界可能是任意非凸形状。因此聚类中心 cluster_center 不一定在簇的内部。因此把聚类中心的 `eps` 邻域的 embedding 归于该簇的方法只是一种近似，实际上并不严谨。

In [16]:
test_df = df[SAMPLE_NUM:]
test_df

,item_id,embeddings
7000,7000,"[-0.008013587445020676, -0.010674460791051388,..."
7001,7001,"[-0.0015732829924672842, -0.018495790660381317..."
7002,7002,"[0.00030177217558957636, -0.012919916771352291..."
7003,7003,"[-0.0028342537116259336, 0.0014691815013065934..."
7004,7004,"[0.002343971747905016, -0.029230637475848198, ..."
...,...,...
9995,9995,"[0.02526906132698059, 0.006334671750664711, 0...."
9996,9996,"[-0.0032427890691906214, 0.0032633657101541758..."
9997,9997,"[0.001930834841914475, -0.025012478232383728, ..."
9998,9998,"[0.025050941854715347, -0.017404677346348763, ..."


In [17]:
# 声明一个新的 id 生成器
cluster_ids = train_df['cluster_id'].tolist()
gen = id_generator(set(cluster_ids))

# 新加入的 embedding 比照现有类心，若这个 embedding 在某个类心的 eps 范围内则加入该簇，如果不在就建立新簇
test_item_id = test_df['item_id'].tolist()
test_embeddings = test_df['embeddings'].tolist()
# for embed in test_embeddings:
for item_id, embed in tqdm(zip(test_item_id, test_embeddings), total=len(test_df), desc="Processing embeddings"):
    found_cluster = False

    cluster_ids = cluster_center_df['cluster_id'].tolist()
    cluster_centers = cluster_center_df['cluster_center'].tolist()
    for cluster_id, cluster_center in zip(cluster_ids, cluster_centers):
        distance = cosine_distances([embed], [cluster_center])[0][0]
        if distance <= eps:
            found_cluster = True
            this_cluster_id = cluster_id
            this_cluster_center = cluster_center
            break
    if not found_cluster:
        this_cluster_id = next(gen)
        this_cluster_center = embed

    # 更新聚类 ID 表
    cluster_id_row = {
        'item_id': item_id,
        'embeddings': embed,
        'cluster_id': this_cluster_id}
    cluster_id_df = pd.concat([cluster_id_df, pd.DataFrame([cluster_id_row])], ignore_index=True)

    # 更新聚类中心表
    if found_cluster:
        # 更新现有聚类中心
        cluster_embeddings = cluster_id_df[cluster_id_df["cluster_id"] == this_cluster_id]["embeddings"].tolist()
        cluster_embeddings += [embed]
        new_cluster_center = np.mean(cluster_embeddings, axis=0).tolist()
        # 将 new_cluster_center 作为一个整体赋值给每一行
        for idx in cluster_center_df.index[cluster_center_df['cluster_id'] == this_cluster_id]:
            cluster_center_df.at[idx, 'cluster_center'] = new_cluster_center
    else:
        # 新建聚类中心
        cluster_center_row = {
            'cluster_id': this_cluster_id,
            'cluster_center': this_cluster_center}
        cluster_center_df = pd.concat([cluster_center_df, pd.DataFrame([cluster_center_row])], ignore_index=True)

Processing embeddings:   0%|          | 0/3000 [00:00<?, ?it/s]

In [18]:
cluster_id_df

,item_id,embeddings,cluster_id
0,0,"[0.024523582309484482, -0.03633105754852295, 0...",148
1,1,"[-0.002521098591387272, 0.022899063304066658, ...",0
2,2,"[0.008400454185903072, -0.012612388469278812, ...",149
3,3,"[-0.004734962247312069, -0.0035224033053964376...",150
4,4,"[-0.021240245550870895, -0.03918471559882164, ...",1
...,...,...,...
9995,9995,"[0.02526906132698059, 0.006334671750664711, 0....",7180
9996,9996,"[-0.0032427890691906214, 0.0032633657101541758...",7181
9997,9997,"[0.001930834841914475, -0.025012478232383728, ...",38
9998,9998,"[0.025050941854715347, -0.017404677346348763, ...",56


In [19]:
cluster_center_df

,cluster_id,cluster_center
0,0,"[-0.0016610274324193597, 0.022706498248645894,..."
1,1,"[-0.0076888806399846524, -0.026927861867339482..."
2,2,"[0.003304169833427295, -0.008476288610836491, ..."
3,3,"[-0.005446118474840962, 0.03490327546993891, 0..."
4,4,"[-0.0017701715878052867, -0.019710551424204935..."
...,...,...
7178,7178,"[-0.027295535430312157, 0.028355276212096214, ..."
7179,7179,"[0.04032472148537636, -0.017514986917376518, 0..."
7180,7180,"[0.02526906132698059, 0.006334671750664711, 0...."
7181,7181,"[-0.0032427890691906214, 0.0032633657101541758..."
